<a href="https://colab.research.google.com/github/ashioyajotham/Natural-Language-Processing/blob/main/Finetuning/Falcon/Fine_Tuning_Falcon_7B_for_Code_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.

In [2]:
from datasets import load_dataset

# Specify the name of the dataset
#dataset_name = "yahma/alpaca-cleaned"
dataset_name = "tatsu-lab/alpaca"


# Load the dataset from the specified name and select the "train" split
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# We will be loading the Falcon 7B model, applying 4bit quantization to it, and then adding LoRA adapters to the model.
import torch

from transformers import FalconForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Defining the name of the Falcon model
model_name = "ybelkada/falcon-7b-sharded-bf16"

# Configuring the BitsAndBytes quantization
bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
device_map = 'auto',
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.float16,
)

# Loading the Falcon model with quantization configuration
model = FalconForCausalLM.from_pretrained(
model_name,
quantization_config=bnb_config,
trust_remote_code=True
)

# Disabling cache usage in the model configuration
model.config.use_cache = False

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
# Load the tokenizer for the Falcon 7B model with remote code trust
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the padding token to be the same as the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

In [5]:
# Import the necessary module for LoRA configuration
from peft import LoraConfig

# Define the parameters for LoRA configuration
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

# Create the LoRA configuration object
peft_config = LoraConfig(
lora_alpha=lora_alpha,
lora_dropout=lora_dropout,
r=lora_r,
bias="none",
task_type="CAUSAL_LM",
target_modules=[
"query_key_value",
"dense",
"dense_h_to_4h",
"dense_4h_to_h",
]
)

In [6]:
from transformers import TrainingArguments
# Define the directory to save training results
output_dir = "./results"

# Set the batch size per device during training
per_device_train_batch_size = 1

# Number of steps to accumulate gradients before updating the model
gradient_accumulation_steps = 4

# Choose the optimizer type (e.g., "paged_adamw_32bit")
optim = "paged_adamw_32bit"

# Interval to save model checkpoints (every 10 steps)
save_steps = 10

# Interval to log training metrics (every 10 steps)
logging_steps = 10

# Learning rate for optimization
learning_rate = 2e-4

# Maximum gradient norm for gradient clipping
max_grad_norm = 0.3

# Maximum number of training steps
max_steps = 50

# Warmup ratio for learning rate scheduling
warmup_ratio = 0.03

# Type of learning rate scheduler (e.g., "constant")
lr_scheduler_type = "constant"

# Create a TrainingArguments object to configure the training process
training_arguments = TrainingArguments(
output_dir=output_dir,
per_device_train_batch_size=per_device_train_batch_size,
gradient_accumulation_steps=gradient_accumulation_steps,
optim=optim,
save_steps=save_steps,
logging_steps=logging_steps,
learning_rate=learning_rate,
fp16=True,  # Use mixed precision training (16-bit)
max_grad_norm=max_grad_norm,
max_steps=max_steps,
warmup_ratio=warmup_ratio,
group_by_length=True,
lr_scheduler_type=lr_scheduler_type,
)

In [7]:
#dataset = dataset.map(lambda x: {"text": x["input"]+x["output"]})

# Import the SFTTrainer from the TRL library
from trl import SFTTrainer

# Set the maximum sequence length
max_seq_length = 512

# Create a trainer instance using SFTTrainer
trainer = SFTTrainer(
model=model,
train_dataset=dataset,
peft_config=peft_config,
dataset_text_field="text",
max_seq_length=max_seq_length,
tokenizer=tokenizer,
args=training_arguments,
)

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [8]:
# Iterate through the named modules of the trainer's model
for name, module in trainer.model.named_modules():

# Check if the name contains "norm"
  if "norm" in name:
	  # Convert the module to use torch.float32 data type
	  module = module.to(torch.float32)


trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.


Step,Training Loss
10,1.570600
20,1.187700
30,1.038300
40,0.877700
50,0.839800


TrainOutput(global_step=50, training_loss=1.1028080558776856, metrics={'train_runtime': 566.3511, 'train_samples_per_second': 0.353, 'train_steps_per_second': 0.088, 'total_flos': 867410874639360.0, 'train_loss': 1.1028080558776856, 'epoch': 0.0})

In [9]:
# Save the model
trainer.save_model("./falcon-7b-code-gen")
#tokenizer.save_pretrained("./tokenizer")

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
from huggingface_hub import whoami

whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

{'type': 'user',
 'id': '6393064837ce40c88b558b7d',
 'name': 'ashioyajotham',
 'fullname': 'Victor Ashioya',
 'email': 'victorashioya960@gmail.com',
 'emailVerified': True,
 'plan': 'NO_PLAN',
 'canPay': False,
 'isPro': False,
 'periodEnd': None,
 'avatarUrl': '/avatars/a824c22e393d4765737b778cf30ade0b.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'pushing', 'role': 'write'}}}

In [ ]:
from huggingface_hub import create_repo

create_repo(repo_id="finetuned-falcon")


In [13]:
trainer.push_to_hub("finetuned-falcon")

adapter_model.bin:   0%|          | 0.00/522M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

'https://huggingface.co/ashioyajotham/results/tree/main/'

In [15]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
sequences = pipeline(
   "Describe an easy recipe for 5 people using rice and chicken",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Describe an easy recipe for 5 people using rice and chicken
I would make a stir fry rice dish using cooked chicken and vegetables such as onion, garlic, and bell peppers with a sweet and sour sauce.
What type of sauce would you use?
Sauce would be a mix of brown sugar, soy sauce, ketchup, vinegar, and garlic.
What side dishes would you serve?
I would serve steamed vegetables or a fresh fruit salad with this dish.


In [19]:
device = "cuda" # the device to load the model onto

#model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
#model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


<|im_start|>user
What is your favourite condiment?<|im_end|>
<|im_start|>assistant
Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!<|im_end|>
<|im_start|>user
Do you have mayonnaise recipes?<|im_end|>
<|im_start|>assistant
Oh, you bet! I have a couple of recipes for amazing homemade mayonnaise. First, for a classic mayonnaise I use a mixture of fresh lemon juice and white wine vinegar. Then, I beat eggs, sugar, and the mayonnaise mix until smooth. The final product tastes like tangy heaven! Second, I recommend trying my dill mayonnaise for an herbaceous, fresh flavour. I use fresh chopped dill and add a dash of lemon juice. Yum!<|im_end|>
<|im_start|>user
What is your favourite condiment?<|im_end|>
<|im_start|>assistant
Hmm, that's a tough one. I really love both mayo and mustard, so it's hard to choose between the two delicious condiments. I guess it'd have to be a tie between the 

In [ ]:
from transformers import pipeline
generator = pipeline('text-generation', model = model, tokenizer=tokenizer)
generator("Hello my name is Ashioya, I'm a data scientist", max_length = 1000, top_k = 5, temperature = .8, do_sample = True, num_return_sequences=1)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},
##  {'generated_text': "Hello, I'm a language modeler. I write and maintain software in Python. I love to code, and that includes coding things that require writing"}, ...


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[{'generated_text': "Hello my name is Ashioya, I'm a data scientist, a writer, an artist, a storyteller, and a lover of all things tech. I've been working with data for over ten years and I'm excited to share my knowledge with you. This is my space to share my thoughts, insights, and knowledge of all things data.\nI'm also a proud mother of two wonderful children, and a wife to an amazing husband. I'm an avid reader, a writer, an artist, a lover of all things tech, and an explorer of all things new. I'm also an avid traveler, and an enthusiastic learner who enjoys learning new things and exploring the world."}]

In [21]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Replace 'model_name' with the name of the model you want to use
model_name = model
model = model
tokenizer = tokenizer

generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

prompt = "Hello my name is Ashioya, I'm a data scientist"
generated_text = generator(prompt, max_length=200, top_k=5, do_sample=True, num_return_sequences=1)

print(generated_text[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Hello my name is Ashioya, I'm a data scientist at a startup called Apertio. I'm also a data science instructor with DataScienceDojo and DataSchool. I'm a former software engineer and have a background in computer science. I've spent over a decade working in the tech industry, primarily in the area of machine learning. I have a passion for helping people understand and apply machine learning in their daily lives. I enjoy working with all types of data and am excited about the potential of AI to transform our world for the better. I am a strong believer that technology is a tool that can help us achieve our goals.


In [ ]:
import torch
from transformers import AutoTokenizer, FalconForCausalLM

tokenizer = tokenizer
model = model

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"  # the device to load the model onto

model = model
tokenizer = tokenizer

question = "Which is the tallest building in the world?"

# Encode the question
question_input = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate an answer
generated_ids = model.generate(question_input["input_ids"], max_length = 100, temperature = 0.7, top_k = 5, do_sample=True, pad_token_id=tokenizer.eos_token_id)

answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Answer:", answer)

Answer: Which is the tallest building in the world?
The Burj Khalifa in Dubai, United Arab Emirates, is the tallest man-made structure in the world. Completed in 2010, it stands at 828 metres (2,717 ft) and has 163 floors..
How many tallest buildings are there in the world?
The Burj Khalifa is the tallest building in the world, standing 829.8 metres (2,722 ft) above the ground.


In [23]:
device = "cuda"  # the device to load the model onto

model = model
tokenizer = tokenizer

# Restructured code generation prompt
code_prompt = """
Write Python code that takes a list of numbers as input and calculates the average of those numbers.
You can assume the input list is named 'numbers'.
The code should return the average as a floating-point number.
Please include necessary variable declarations, input validation, and comments for clarity.
"""

# Encode the code generation prompt
prompt_input = tokenizer(code_prompt, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate Python code
generated_ids = model.generate(prompt_input["input_ids"], max_length=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)

generated_code = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Generated Python Code:")
print(generated_code)

Generated Python Code:

Write Python code that takes a list of numbers as input and calculates the average of those numbers.
You can assume the input list is named 'numbers'.
The code should return the average as a floating-point number.
Please include necessary variable declarations, input validation, and comments for clarity.
Solution:
numbers = [1, 10, 20] print("Average= ", sum(numbers)/len(numbers))
Explanation:
sum() is used to find the total sum of all numbers in the list.
We'll then divide the result by the total number of lists.
As a result, the average is 10.
print("Average= ", sum(numbers)/len(numbers))
